In [170]:
import numpy as np
import pandas as pd
import os

In [247]:
#Set the paths
registered_business_path = os.path.join('Cleaned Datasets', 'cleaned_registered_food_services.csv')
health_scores_path = os.path.join('Cleaned Datasets', 'health_scores.csv')

geocoded_registered_business_path = os.path.join('Cleaned Datasets', 'addresses-2018.csv')
geocoded_health_scores_path = os.path.join('Cleaned Datasets', 'geocoded_health_scores_addresses-2018.csv')

#Read in the csvs
registered_business_df = pd.read_csv(registered_business_path)
health_scores_df = pd.read_csv(health_scores_path)

geocoded_rb_df = pd.read_csv(geocoded_registered_business_path)
geocoded_hs_df = pd.read_csv(geocoded_health_scores_path)

In [254]:
registered_business_df = registered_business_df[registered_business_df['business_end_date'].notnull()]
list_a = registered_business_df['business_name'].unique()
df = pd.DataFrame(list_a)
df.to_csv('unique.csv', index=False)

In [237]:
geocoded_hs_df = geocoded_hs_df.drop_duplicates(subset=['google_place_id'], keep='first').reset_index(drop=True)

geocoded_hs_df = geocoded_hs_df.dropna(subset=['formatted_address'])

len(geocoded_hs_df)

path = os.path.join('Cleaned Datasets', 'test.csv')
geocoded_hs_df.to_csv(path, index=False)

In [173]:
geocoded_hs_df.head()

,accuracy,formatted_address,google_place_id,input_string,latitude,longitude,number_of_results,postcode,status,type
0,ROOFTOP,"1200 Van Ness Ave 3RD FLOOR, San Francisco, CA...",EjozUkQgRkxPT1IsIDEyMDAgVmFuIE5lc3MgQXZlLCBTYW...,"1200 VAN NESS AVE, 3RD FLOOR",37.786777,-122.421335,1,94109.0,OK,subpremise
1,ROOFTOP,"500 California St 2ND FLOOR, San Francisco, C...",EjsyTkQgIEZMT09SLCA1MDAgQ2FsaWZvcm5pYSBTdCwgU2...,"500 CALIFORNIA ST, 2ND FLOOR",37.793123,-122.403101,1,94104.0,OK,subpremise
2,ROOFTOP,"2801 Leavenworth St, San Francisco, CA 94133, USA",ChIJu2t7buGAhYARabsFEzVXIqw,2801 LEAVENWORTH ST,37.807021,-122.419384,2,94133.0,OK,premise
3,ROOFTOP,"3202 Folsom St, San Francisco, CA 94110, USA",ChIJARWKcUV-j4ARfAqXO018-80,3202 FOLSOM ST,37.747104,-122.413831,2,94110.0,OK,premise
4,ROOFTOP,"747 Irving St, San Francisco, CA 94122, USA",ChIJE9-qSVyHhYARqRpbrJ9rHlA,747 IRVING ST,37.763933,-122.466047,2,94122.0,OK,street_address


In [239]:
#Merge the registered business df with the geocodes to get unique location id
merge_registered_business_df = registered_business_df.merge(geocoded_rb_df, left_on='street_address', right_on='input_string')
# merge_registered_business_df.columns.values

path = os.path.join('Cleaned Datasets', 'matched.csv')
merge_registered_business_df.to_csv(path, index=False)

len(merge_registered_business_df)


3500

In [240]:
#Get columns we need
merge_registered_business_df = merge_registered_business_df[['location_id', 'ownership_name', 'business_name', 'category',
                            'lic_code', 'street_address', 'State', 'City', 'zipcode', 
                            'business_start_date', 'business_end_date', 'neighborhoods', 'business_location',
                            'supervisor_district', 'Lifespan', 'formatted_address', 'google_place_id', 'latitude', 'longitude']]

merge_registered_business_df.head(20)

#Add a new column to filter out first word in restaurants' name
merge_registered_business_df['formatted_name'] = merge_registered_business_df['business_name'].apply(lambda x: x.split(' ', 1)[0].lower())

path = os.path.join('Cleaned Datasets', 'matched.csv')
merge_registered_business_df.to_csv(path, index=False)

In [241]:
#Merge the healthscores  df with the geocodes to get unique location id
merge_health_scores_df = health_scores_df.merge(gecoded_hs_df, left_on='address', right_on='input_string')

#Drop the rows where formatted address is null
merge_health_scores_df = merge_health_scores_df.dropna(subset=['formatted_address'])
len(merge_health_scores_df)


3914

In [242]:
merge_health_scores_df.head()
merge_health_scores_df.columns.values
len(merge_health_scores_df)

3914

In [243]:
#get the columns we need
merge_health_scores_df = merge_health_scores_df[['business_id', 'name', 'score', 'year', 'formatted_address']]

#drop the restaurants who are in the same location
#merge_health_scores_df = merge_health_scores_df.drop_duplicates(subset=['formatted_address', 'year'], keep=False)

#drop restaurants who don't have a health score
merge_health_scores_df = merge_health_scores_df.dropna(subset=['score'])

#Add a new column to filter out first word in restaurants' name
merge_health_scores_df['formatted_name'] = merge_health_scores_df['name'].apply(lambda x: x.split(' ', 1)[0].lower())


path = os.path.join('Cleaned Datasets', 'health_matched.csv')
merge_health_scores_df.to_csv(path, index=False)

In [244]:
#Merge the two files to see if there are any matches
merged_df = merge_registered_business_df.merge(merge_health_scores_df, left_on=['formatted_address', 'formatted_name'], right_on=['formatted_address', 'formatted_name'] )
len(merged_df)
# merged_df.head(20)

1257

In [245]:
merged_df.head(30)

,location_id,ownership_name,business_name,category,lic_code,street_address,State,City,zipcode,business_start_date,...,Lifespan,formatted_address,google_place_id,latitude,longitude,formatted_name,business_id,name,score,year
0,0343504-01-001,East-West Holdings Inc,Burma Super Star Restaurant,Food Services,"Restaurant 1,000 - 2,000 Sqft",309 Clement St,CA,San Francisco,94118.0,03/01/2001,...,NaN,"309 Clement St, San Francisco, CA 94118, USA",ChIJ7Tyh4TuHhYARdYpz0QD1nCk,37.782818,-122.462536,burma,1977,BURMA SUPERSTAR RESTAURANT,91.0,2017.0
1,0343504-01-001,East-West Holdings Inc,Burma Super Star Restaurant,Food Services,"Restaurant 1,000 - 2,000 Sqft",309 Clement St,CA,San Francisco,94118.0,03/01/2001,...,NaN,"309 Clement St, San Francisco, CA 94118, USA",ChIJ7Tyh4TuHhYARdYpz0QD1nCk,37.782818,-122.462536,burma,1977,BURMA SUPERSTAR RESTAURANT,87.0,2016.0
2,1175523-01-181,Patrick John Alvir,Baby's Eatery And Palabok,Food Services,NaN,4609 Mission Street,CA,San Francisco,94112.0,01/08/2018,...,NaN,"4609 Mission St, San Francisco, CA 94112, USA",ChIJ__-iUo9-j4ARLzkmCcZcBmg,37.724433,-122.434725,baby's,824,BABY'S EATERY AND PALABOK,77.0,2018.0
3,1175523-01-181,Patrick John Alvir,Baby's Eatery And Palabok,Food Services,NaN,4609 Mission Street,CA,San Francisco,94112.0,01/08/2018,...,NaN,"4609 Mission St, San Francisco, CA 94112, USA",ChIJ__-iUo9-j4ARLzkmCcZcBmg,37.724433,-122.434725,baby's,824,BABY'S EATERY AND PALABOK,76.0,2016.0
4,1175523-01-181,Patrick John Alvir,Baby's Eatery And Palabok,Food Services,NaN,4609 Mission Street,CA,San Francisco,94112.0,01/08/2018,...,NaN,"4609 Mission St, San Francisco, CA 94112, USA",ChIJ__-iUo9-j4ARLzkmCcZcBmg,37.724433,-122.434725,baby's,824,BABY'S EATERY AND PALABOK,80.0,2016.0
5,1175523-01-181,Patrick John Alvir,Baby's Eatery And Palabok,Food Services,NaN,4609 Mission Street,CA,San Francisco,94112.0,01/08/2018,...,NaN,"4609 Mission St, San Francisco, CA 94112, USA",ChIJ__-iUo9-j4ARLzkmCcZcBmg,37.724433,-122.434725,baby's,824,BABY'S EATERY AND PALABOK,71.0,2017.0
6,0364643-06-003,Sodexo America Llc,Academy Of Arts Univ,Food Services,"Restaurant Greater Than 2,000 Sqft",860 Sutter St,CA,San Francisco,94102.0,11/18/2002,...,NaN,"860 Sutter St, San Francisco, CA 94109, USA",ChIJOZXlO5KAhYARLUUIJadgltY,37.788769,-122.414608,academy,2920,ACADEMY OF ARTS UNIV- I HOUSE CAFE,100.0,2016.0
7,0364643-06-003,Sodexo America Llc,Academy Of Arts Univ,Food Services,"Restaurant Greater Than 2,000 Sqft",860 Sutter St,CA,San Francisco,94102.0,11/18/2002,...,NaN,"860 Sutter St, San Francisco, CA 94109, USA",ChIJOZXlO5KAhYARLUUIJadgltY,37.788769,-122.414608,academy,2920,ACADEMY OF ARTS UNIV- I HOUSE CAFE,96.0,2017.0
8,0364643-06-003,Sodexo America Llc,Academy Of Arts Univ,Food Services,"Restaurant Greater Than 2,000 Sqft",860 Sutter St,CA,San Francisco,94102.0,11/18/2002,...,NaN,"860 Sutter St, San Francisco, CA 94109, USA",ChIJOZXlO5KAhYARLUUIJadgltY,37.788769,-122.414608,academy,2920,ACADEMY OF ARTS UNIV- I HOUSE CAFE,100.0,2018.0
9,1176824-01-181,Mmr-Sf.llc,Marrakech Moroccan Restaurant,Food Services,NaN,419 Ofarrell St,CA,Sanfrancisco,94102.0,01/24/2018,...,NaN,"419 O'Farrell St, San Francisco, CA 94102, USA",ChIJh4AEz4-AhYARXsIEprl4cwk,37.785855,-122.411761,marrakech,3800,MARRAKECH MORROCAN RESTAURANT,88.0,2016.0


In [246]:
path = os.path.join('Cleaned Datasets', 'final-merged.csv')
merged_df.to_csv(path, index=False)